Module 5 — Building a RAG Pipeline (LangChain)
Objectives
- User Query → Embed → Search vector DB → Feed to LLM
- Prompt templates for concise, source-backed answers
- Two approaches:
1) Custom LCEL RAG chain
2) LangChain RetrievalQA chain
- Include citations with metadata (e.g., [doc_id p.43])
Prereqs
- You built an index in Module 3 at ./data/indexes/my_corpus
- Set an LLM key (prefer Gemini):
- export GOOGLE_API_KEY=...
- Optional fallback: OpenAI (export OPENAI_API_KEY=...), but this notebook defaults to Gemini.

Install dependencies (if needed)
- langchain, core, community, google-genai client
- sentence-transformers (for MiniLM query embeddings)
- faiss-cpu
- tqdm

In [12]:
%pip install -q langchain langchain-core langchain-community langchain-google-genai google-generativeai
%pip install -q sentence-transformers faiss-cpu tqdm
import os
import sys
import json
from pathlib import Path
from typing import Any, Dict, List, Tuple, Optional
import numpy as np
from tqdm import tqdm
# FAISS import
try:
    import faiss  # noqa: F401
except Exception:
    import faiss_cpu as faiss  # type: ignore
print(f"Python: {sys.version.split()[0]} | FAISS ok")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Python: 3.12.9 | FAISS ok



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Load the saved FAISS index and docstore (from Module 3)
- We rely on the manifest to detect which embedding backend was used.
- Important: Query embeddings MUST use the same backend/dimension as the index.

In [13]:
INDEX_DIR = Path("./../data/indexes/my_corpus")
assert (INDEX_DIR / "index.faiss").exists(), f"FAISS index not found at {INDEX_DIR}. Please run Module 3."

def load_index(in_dir: Path) -> Tuple[faiss.Index, np.ndarray, List[Dict[str, Any]], Dict[str, Any]]:
    index = faiss.read_index(str(in_dir / "index.faiss"))
    vectors = np.load(in_dir / "vectors.npy")
    docstore = []
    with (in_dir / "docstore.jsonl").open("r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                docstore.append(json.loads(line))
    manifest = json.loads((in_dir / "manifest.json").read_text(encoding="utf-8"))
    return index, vectors, docstore, manifest

index, vectors, store, manifest = load_index(INDEX_DIR)
print(f"Loaded index: dim={vectors.shape[1]}, vectors={vectors.shape[0]}, backend={manifest.get('backend')}")

Loaded index: dim=768, vectors=19, backend=gemini


Embedding backend for queries
- MiniLM (local, free) or Gemini (cloud).
- Must match the backend used when the index was built (see manifest.json).
- If manifest says 'gemini', you need GOOGLE_API_KEY set.

In [14]:
from dotenv import load_dotenv
load_dotenv(r"C:\ML\LU-LiveClasses\DocumentAI\.env")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
BACKEND = manifest.get("backend", "minilm").lower()
print("Backend from manifest:", BACKEND)

class MiniLMBackend:
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        from sentence_transformers import SentenceTransformer
        import torch
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer(model_name, device=device)
        self.dim = self.model.get_sentence_embedding_dimension()
        self.model_name = model_name
    def encode(self, texts: List[str], batch_size: int = 64) -> np.ndarray:
        return self.model.encode(texts, batch_size=batch_size, convert_to_numpy=True, normalize_embeddings=False).astype(np.float32)

class GeminiBackend:
    def __init__(self, model_name: str = "text-embedding-004", api_key: Optional[str] = None):
        assert api_key, "GOOGLE_API_KEY required for Gemini embeddings."
        import google.generativeai as genai
        genai.configure(api_key=api_key)
        self.genai = genai
        self.model_name = model_name
        self.dim = 768
    def encode(self, texts: List[str], batch_size: int = 16) -> np.ndarray:
        vecs: List[List[float]] = []
        for i in tqdm(range(0, len(texts), batch_size), desc="Gemini embed"):
            for t in texts[i:i+batch_size]:
                resp = self.genai.embed_content(model=self.model_name, content=t[:4000])
                vecs.append(resp["embedding"])
        return np.asarray(vecs, dtype=np.float32)

def l2_normalize(vecs: np.ndarray) -> np.ndarray:
    norms = np.linalg.norm(vecs, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    return vecs / norms

if BACKEND == "minilm":
    query_backend = MiniLMBackend()
elif BACKEND == "gemini":
    query_backend = GeminiBackend(api_key=GOOGLE_API_KEY)
else:
    raise ValueError("Unknown backend in manifest. Expected 'minilm' or 'gemini'.")

assert query_backend.dim == vectors.shape[1], f"Dim mismatch: backend {query_backend.dim} vs index {vectors.shape[1]}"

Backend from manifest: gemini


Build a LangChain retriever around the existing FAISS index
- We wrap our FAISS + vectors + docstore in a custom BaseRetriever.
- Returns LangChain Documents with page_content and metadata for citations.
- Optional MMR re-ranking for diversity.

In [15]:
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from pydantic import BaseModel, Field
import numpy as np

def embed_query(q: str) -> np.ndarray:
    v = query_backend.encode([q], batch_size=1)
    return l2_normalize(v)

def search_faiss(index: faiss.Index, qvec: np.ndarray, top_k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
    D, I = index.search(qvec, top_k)
    return D[0], I[0]

def mmr_rerank(query_vec: np.ndarray,
               candidates_idx: np.ndarray,
               candidate_vecs: np.ndarray,
               top_k: int = 5,
               lambda_mult: float = 0.5) -> List[int]:
    selected: List[int] = []
    pool = candidates_idx.tolist()
    sims_q = (candidate_vecs @ query_vec[0])
    while len(selected) < min(top_k, len(pool)):
        best_i = None
        best_score = -1e9
        for idx in pool:
            rel = sims_q[idx]
            if not selected:
                score = rel
            else:
                div = max(candidate_vecs[idx] @ candidate_vecs[j] for j in selected)
                score = lambda_mult * rel - (1 - lambda_mult) * div
            if score > best_score:
                best_score = score
                best_i = idx
        selected.append(best_i)
        pool.remove(best_i)
    return selected

class FaissJSONLRetriever(BaseRetriever, BaseModel):
    index: Any
    vectors: np.ndarray
    store: List[Dict[str, Any]]
    top_k: int = 5
    use_mmr: bool = False
    mmr_lambda: float = 0.5
    fetch_k: int = 25  # number to fetch before MMR
    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str) -> List[Document]:
        qv = embed_query(query)
        top_k = self.top_k
        if self.use_mmr:
            D, I = search_faiss(self.index, qv, self.fetch_k)
            selected = mmr_rerank(qv, I, self.vectors, top_k=top_k, lambda_mult=self.mmr_lambda)
            I = np.array(selected, dtype=int)
        else:
            D, I = search_faiss(self.index, qv, top_k)
        docs: List[Document] = []
        for idx in I[:top_k]:
            rec = self.store[idx]
            docs.append(Document(page_content=rec["text"], metadata=rec.get("metadata", {})))
        return docs

retriever = FaissJSONLRetriever(index=index, vectors=vectors, store=store, top_k=5, use_mmr=True, mmr_lambda=0.5, fetch_k=25)

LLM setup (Gemini preferred)
- Uses ChatGoogleGenerativeAI (gemini-1.5-flash) for fast, cost-effective answers.
- If GOOGLE_API_KEY is missing, you can switch to ChatOpenAI by setting OPENAI_API_KEY and adjusting the code.

In [16]:
USE_GEMINI = bool(GOOGLE_API_KEY)

if USE_GEMINI:
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2, max_output_tokens=512)
    print("Using Gemini for answers.")
else:
    # Optional fallback to OpenAI if desired:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
    if OPENAI_API_KEY:
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
        print("Using OpenAI fallback. Set GOOGLE_API_KEY to use Gemini.")
    else:
        raise EnvironmentError("No LLM configured. Set GOOGLE_API_KEY (preferred) or OPENAI_API_KEY.")

Using Gemini for answers.


Approach 1: Custom LCEL RAG chain
- Format retrieved documents into a compact context with citations.
- Prompt instructs the model to cite sources like [doc_id p.page].
- Returns a concise, source-backed answer.

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs: List[Document]) -> str:
    lines = []
    for d in docs:
        m = d.metadata
        cite = f"{m.get('doc_id','?')} p.{m.get('page','?')}"
        lines.append(f"[{cite}] {d.page_content}")
    return "\n\n".join(lines)

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a concise, domain-agnostic assistant. Use ONLY the provided context to answer.\n"
     "Always cite sources in brackets like [doc_id p.page]. If unsure, say you don't know.\n"
     "Be precise and avoid speculation."),
    ("human",
     "Question: {question}\n\nContext:\n{context}\n\nAnswer:")
])

rag_chain = (
    {"context": retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Test the custom RAG chain
- Try queries relevant to your corpus (e.g., contracts: termination, confidentiality).
- The answer should include bracketed citations.

In [21]:
queries = [
    "Can heart beat outside body?",
    "what is full form of HDL?",
    "Impact of lifestyle and weight on heart?"
]

for q in queries:
    print(f"\nQ: {q}")
    ans = rag_chain.invoke(q)
    print(ans)


Q: Can heart beat outside body?


Gemini embed: 100%|██████████| 1/1 [00:00<00:00,  1.79it/s]


Yes, a heart can beat outside the body as long as it receives oxygen [facts p.None].

Q: what is full form of HDL?


Gemini embed: 100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


HDL stands for high-density lipoprotein [heart-health p.1].

Q: Impact of lifestyle and weight on heart?


Gemini embed: 100%|██████████| 1/1 [00:00<00:00,  1.46it/s]


Obesity and being overweight are risk factors for heart disease [heart-health p.1].  Maintaining a healthy weight can reduce heart disease risk [heart-health p.1, How-to-Monitor-Cholesterol-BP-Weight p.2].  Lifestyle changes, including increased physical activity (at least 150 minutes of moderate-intensity aerobic activity per week) and reducing caloric intake, can help with weight loss [How-to-Monitor-Cholesterol-BP-Weight p.2].  Even a 3-5% weight loss can offer health benefits, with greater benefits seen with larger weight losses (5-10%) [How-to-Monitor-Cholesterol-BP-Weight p.2].


Approach 2: LangChain RetrievalQA chain
- Uses the same retriever, but leverages the built-in RetrievalQA helper.
- We pass a custom prompt to enforce citations.

In [25]:
from langchain.chains import RetrievalQA

qa_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a concise assistant. Use ONLY the retrieved context to answer.\n"
     "Cite sources in brackets like [doc_id p.page]. If the answer is not in the context, say you don't know."),
    ("human",
     "Question: {question}\n\nRetrieved context:\n{context}\n\nAnswer:")
])

# The RetrievalQA 'stuff' chain passes a {context} string of concatenated docs.
# We'll enforce our own formatting by wrapping the retriever with a small adapter.
def retriever_to_context(query: str) -> str:
    # LCEL adapter to get docs and format them
    docs = retriever.get_relevant_documents(query)
    return format_docs(docs)

retrievalqa_chain = (
    {"context": RunnableLambda(retriever_to_context), "question": RunnablePassthrough()}
    | qa_prompt
    | llm
    | StrOutputParser()
)

for q in ["How to Lower cholestrol?", "What is Spandan?"]:
    print(f"\nQ: {q}")
    ans = retrievalqa_chain.invoke(q)
    print(ans)


Q: How to Lower cholestrol?


Gemini embed: 100%|██████████| 1/1 [00:00<00:00,  2.21it/s]


To lower cholesterol, eat a diet rich in whole grains, plant-based proteins, fish, nontropical vegetable oils, nuts, and seeds [How-to-Monitor-Cholesterol-BP-Weight p.2].  Limit sodium, sweets, sugar-sweetened beverages, processed meats, trans fats, and refined carbohydrates [How-to-Monitor-Cholesterol-BP-Weight p.2]. Eat at least two servings of fatty fish per week [How-to-Monitor-Cholesterol-BP-Weight p.2]. Consume less than 1,500 mg of sodium daily [How-to-Monitor-Cholesterol-BP-Weight p.2]. Limit alcohol consumption [How-to-Monitor-Cholesterol-BP-Weight p.2].  Be physically active [How-to-Monitor-Cholesterol-BP-Weight p.2]. Maintain a healthy weight [How-to-Monitor-Cholesterol-BP-Weight p.2], don't smoke, and take prescribed medications [How-to-Monitor-Cholesterol-BP-Weight p.2].

Q: What is Spandan?


Gemini embed: 100%|██████████| 1/1 [00:00<00:00,  2.10it/s]


Based on the provided text, Spandan is a portable ECG. [spandan p.None]


Post-processing: Extract and display cited sources explicitly
- Pull out top-k docs used to build the answer so you can show clickable citations in a UI.

In [28]:
def get_sources_for_query(query: str, k: int = 5) -> List[Dict[str, Any]]:
    docs = retriever.get_relevant_documents(query)
    out = []
    for d in docs[:k]:
        m = d.metadata
        out.append({
            "doc_id": m.get("doc_id"),
            "page": m.get("page"),
            "section_title": m.get("section_title"),
            "source_path": m.get("source_path")
        })
    return out

q = "how to prevent heart disease?"
sources = get_sources_for_query(q, k=3)
print("\nSources used:")
for s in sources:
    print(f"- {s['doc_id']} p.{s['page']} | section={s['section_title']} | {s['source_path']}")

Gemini embed: 100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


Sources used:
- heart-health p.1 | section=None | ..\data\pdfs\heart-health.pdf
- spandan p.None | section=None | ..\data\images\spandan.png
- spandan_facts p.None | section=None | ..\data\images\spandan_facts.png


Mini Task
1) Ensure your FAISS index from Module 3 exists at ./data/indexes/my_corpus.
2) Set GOOGLE_API_KEY (preferred) or OPENAI_API_KEY.
3) Run:
- Build retriever
- Custom LCEL RAG chain tests
- RetrievalQA chain tests
4) Verify answers are concise and include citations like [doc_id p.page].

Tips and notes
- Prompting: Keep the instruction explicit about using only provided context and citing sources.
- Chunking overlap (Module 2) helps provide enough context per chunk to answer questions cleanly.
- Top-k tuning: Start with k=4–6; increase if answers frequently say “don’t know.”
- MMR: Improves diversity when many near-duplicate chunks exist. Trade-off with recall.
- Hallucinations: Use low temperature and strong instructions; consider adding “If any part is uncertain, state the uncertainty.”
- Token control: If contexts are large, truncate each chunk to a max length or use a reranker to keep only the most relevant passages.
- Mixing sources: If you added OCR chunks in Module 4, they’re seamlessly retrievable with the same retriever/index (as long as the embedding backend matches).

Next steps (Module 6)
- Add conversational memory with LangChain’s ConversationBufferMemory.
- Persist per-session history and support follow-up questions that reference prior answers.
- Introduce tool-using agent patterns for multi-step reasoning (e.g., retrieval + calculator + web search).